# Merge Dask DataFrames

## Merge Large Dask DataFrame to Small Pandas DataFrame

In [1]:
import coiled

In [2]:
import pandas as pd
import dask.dataframe as dd
import dask

In [3]:
cluster = coiled.Cluster(
    name="dask-merge",
    n_workers=50,
    backend_options={'spot':'True'},
    software='coiled-examples/numpy-zarr',
)

/Users/rpelgrim/mambaforge/envs/numpy-zarr/lib/python3.9/site-packages/rich/live.py:221: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Using existing cluster: 'dask-merge'

In [4]:
from distributed import Client
client = Client(cluster)
client

/Users/rpelgrim/mambaforge/envs/numpy-zarr/lib/python3.9/site-packages/distributed/client.py:1131: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2021.11.2     | 2021.11.1     | 2021.11.1     |
| distributed | 2021.11.2     | 2021.11.1     | 2021.11.1     |
| lz4         | None          | 3.1.3         | 3.1.3         |
| msgpack     | 1.0.3         | 1.0.2         | 1.0.2         |
| numpy       | 1.21.4        | 1.21.2        | 1.21.2        |
| pandas      | 1.3.5         | 1.3.4         | 1.3.4         |
| python      | 3.9.9.final.0 | 3.9.6.final.0 | 3.9.6.final.0 |
+-------------+---------------+---------------+---------------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tls://10.4.4.132:8786' processes=50 threads=100, memory=383.35 GiB>

In [5]:
client.restart()

distributed.utils - ERROR - 'tls://10.4.1.96:41995'
Traceback (most recent call last):
  File "/Users/rpelgrim/mambaforge/envs/numpy-zarr/lib/python3.9/site-packages/distributed/utils.py", line 653, in log_errors
    yield
  File "/Users/rpelgrim/mambaforge/envs/numpy-zarr/lib/python3.9/site-packages/distributed/deploy/cluster.py", line 196, in _watch_worker_status
    self._update_worker_status(op, msg)
  File "/Users/rpelgrim/mambaforge/envs/numpy-zarr/lib/python3.9/site-packages/distributed/deploy/cluster.py", line 206, in _update_worker_status
    del self.scheduler_info["workers"][msg]
KeyError: 'tls://10.4.1.96:41995'


<Client: 'tls://10.4.4.132:8786' processes=0 threads=0, memory=0 B>

In [9]:
large = dask.datasets.timeseries(start="2000", end="2005", freq="1H")
small = dask.datasets.timeseries(start="2000", end="2005", freq="1D", dtypes={"z": int})

In [10]:
large.dtypes

id        int64
name     object
x       float64
y       float64
dtype: object

In [11]:
large

,id,name,x,y
npartitions=1827,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2004-12-31,...,...,...,...
2005-01-01,...,...,...,...


In [13]:
large.memory_usage().compute()

Index    350784
id       350784
name     350784
x        350784
y        350784
dtype: int64

In [14]:
large.to_parquet(
    's3://coiled-datasets/dask-merge/large.parquet',
    engine="pyarrow"
)

KilledWorker: ("('reset_index-f62e4e433c74fe34223d80ef584137e5', 388)", <WorkerState 'tls://10.4.10.229:39433', name: coiled-dask-rrpelgr71-97896-worker-4a9861ed52, status: closed, memory: 0, processing: 9>)

In [4]:
small = small.compute()

In [5]:
small

,z
timestamp,
2000-01-01,984
2000-01-02,1021
2000-01-03,979
2000-01-04,1050
2000-01-05,1003
2000-01-06,1009
2000-01-07,984
2000-01-08,979
2000-01-09,977


In [6]:
result = large.merge(small, how="left", on=["timestamp"])

In [7]:
result.head()

,id,name,x,y,z
timestamp,,,,,
2000-01-01 00:00:00,1021,Laura,-0.198011,-0.835659,984.0
2000-01-01 00:00:10,993,George,-0.711381,0.904073,NaN
2000-01-01 00:00:20,915,Laura,0.807961,-0.473370,NaN
2000-01-01 00:00:30,953,Ray,0.120000,0.743861,NaN
2000-01-01 00:00:40,979,Tim,-0.104925,0.134858,NaN


## Merge two Dask DataFrames

In [8]:
left = dask.datasets.timeseries(dtypes={"foo": int})
right_one = dask.datasets.timeseries(dtypes={"bar": int})
right_two = dask.datasets.timeseries(dtypes={"baz": int})

In [9]:
# timeseries returns a dataframe indexed by
# timestamp, we don't need to set_index.

# left.set_index("timestamp")

#left.to_parquet("left", overwrite=True)
#left = dd.read_parquet("left")

# If the dataframe can fit in RAM, you can also use persist

# left = left.persist()


result1 = left.merge(
    right_one, how="left", left_index=True, right_index=True)

In [13]:
result2 = result1.merge(
    right_two, how="left", left_index=True, right_index=True)

In [11]:
left.head()

,foo
timestamp,
2000-01-01 00:00:00,1032
2000-01-01 00:00:01,993
2000-01-01 00:00:02,993
2000-01-01 00:00:03,995
2000-01-01 00:00:04,995


In [10]:
result1.head()

,foo,bar
timestamp,,
2000-01-01 00:00:00,1032,1016
2000-01-01 00:00:01,993,949
2000-01-01 00:00:02,993,1004
2000-01-01 00:00:03,995,1023
2000-01-01 00:00:04,995,1042


In [14]:
result2.head()

,foo,bar,baz
timestamp,,,
2000-01-01 00:00:00,1032,1016,1014
2000-01-01 00:00:01,993,949,1010
2000-01-01 00:00:02,993,1004,984
2000-01-01 00:00:03,995,1023,1005
2000-01-01 00:00:04,995,1042,981


## Simple Example

In [35]:
import dask.dataframe as dd
import pandas as pd
df_large = pd.DataFrame(
    {
        "Name": ["Azza", "Brandon", "Cedric", "Devonte", "Eli", "Fabio"], 
        "Age": [29, 30, 21, 57, 32, 19]
    }
)
large = dd.from_pandas(df_large, npartitions=2)

In [36]:
small = pd.DataFrame(
    {
        "Name": ["Azza", "Cedric", "Fabio"], 
        "City": ["Beirut", "Dublin", "Rosario"]
    }
)

In [40]:
join = large.merge(small, how="left", on=["Name"])

In [41]:
join.compute()

,Name,Age,City
0,Azza,29,Beirut
1,Brandon,30,NaN
2,Cedric,21,Dublin
0,Devonte,57,NaN
1,Eli,32,NaN
2,Fabio,19,Rosario
